# Spark assignment

Taratukhin Oleg, C41323

Initial env setup:

In [20]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pathlib import Path
import emoji

spark = SparkSession.builder \
    .appName("itmo spark assignment") \
    .master("local[*]") \
    .getOrCreate()

spark

In [28]:
!cd data && ls
!rm -rf results/task4/

PATH_TO_DATA = Path("data/")
PATH_TO_RESULTS = Path("results/task4/")

followers.parquet              posts_api.json
followers_posts_api_final.json posts_likes.parquet
followers_posts_likes.parquet


Utilities:

In [22]:
def save_as_json(df: pyspark.sql.dataframe.DataFrame, name: str):
    df.write \
        .format("json") \
        .save(str(PATH_TO_RESULTS / name))

In [23]:
posts_api = spark \
    .read \
    .json(str(PATH_TO_DATA / "posts_api.json"))

posts_api.limit(10).toPandas()

,attachments,comments,copy_history,copyright,date,from_id,geo,id,key,likes,marked_as_ads,owner_id,post_source,post_type,reposts,signer_id,text,unavailable,views
0,None,"(1, 0, True)","[([Row(album=None, audio=None, doc=None, link=...",None,1418838050,-94,None,17274,-94_17274,"(1, 1, 12, 0)",0,-94,"(None, iphone, api)",post,"(5, 0)",None,,None,None
1,"[(None, None, None, None, None, (791d86f8b302d...","(1, 0, True)",None,None,1418812223,-94,None,17273,-94_17273,"(1, 1, 7, 0)",0,-94,"(None, None, vk)",post,"(3, 0)",None,[Технологические брокеры объединят науку и биз...,None,None
2,None,"(1, 0, True)","[([Row(album=None, audio=None, doc=None, link=...",None,1418932825,-94,None,17313,-94_17313,"(1, 1, 14, 0)",0,-94,"(None, None, vk)",post,"(4, 0)",None,,None,None
3,"[(None, None, None, None, None, (912f07ffb75c8...","(1, 0, True)",None,None,1418929134,-94,None,17301,-94_17301,"(1, 1, 30, 0)",0,-94,"(None, None, vk)",post,"(17, 0)",None,"Дорогие друзья, мы открываем новую рубрику, в ...",None,None
4,"[(None, None, None, None, None, (48de0a7227ce0...","(1, 2, True)",None,None,1418922045,-94,None,17295,-94_17295,"(1, 1, 10, 0)",0,-94,"(None, None, vk)",post,"(0, 0)",None,[Первый выпуск Молодежного коворкинг-центра Ун...,None,None
5,"[(None, None, None, None, None, (01786517929f7...","(1, 3, True)",None,None,1418906160,-94,None,17293,-94_17293,"(1, 1, 51, 0)",0,-94,"(None, None, vk)",post,"(9, 0)",None,[Александр Ишевский о новых продуктах и востре...,None,None
6,"[(None, None, None, None, None, (108e8d387cd05...","(1, 27, True)",None,None,1418893664,-94,None,17277,-94_17277,"(1, 1, 16, 0)",0,-94,"(None, None, vk)",post,"(0, 0)",None,Для желающих увидеть большую пресс-конференцию...,None,None
7,"[(None, None, None, None, None, (c243da0b8a371...","(1, 0, True)",None,None,1418889630,-94,None,17276,-94_17276,"(1, 1, 4, 0)",0,-94,"(None, None, vk)",post,"(0, 0)",None,[Профессоры Пауль Урбах и Юрки Сааринен расска...,None,None
8,"[(None, None, None, None, None, (f5a6642309dea...","(1, 23, True)",None,None,1418997628,-94,None,17326,-94_17326,"(1, 1, 68, 0)",0,-94,"(None, None, vk)",post,"(34, 0)",None,"Подготовка к праздникам уже в самом разгаре, а...",None,None
9,"[(None, None, None, None, None, (b9c7483cdcc2b...","(1, 0, True)",None,None,1418993264,-94,None,17323,-94_17323,"(1, 1, 1, 0)",0,-94,"(None, None, vk)",post,"(1, 0)",None,[Татьяна Богомазова: какой должна быть информа...,None,None


## Task 4

Find emoticons in posts and post comments (negative, positive, neutral) (you can use external libraries or predefined emoticon lists) (use the spark udf and broadcast function for emoticons)

In [29]:
schema = ArrayType(StringType(), containsNull=False)
only_emoji = lambda s: (c for c in s if c in emoji.UNICODE_EMOJI)

emoji_in_posts = posts_api \
    .filter(col('text').isNotNull()) \
    .withColumn("emoji", udf(lambda s: list(only_emoji(s)), schema)("text")) \
    .select("id", "emoji") \
    .filter(size("emoji") > 0)

save_as_json(emoji_in_posts, "task4_emoji.json")

emoji_in_posts.limit(10).toPandas()

,id,emoji
0,17381,"[🎄, 🎄]"
1,42270,[📸]
2,42273,[⚡]
3,42281,[🚀]
4,42292,[🔮]
5,42298,[🚀]
6,42302,[🏕]
7,42313,[💫]
8,42315,[♥]
9,42316,[🔥]
